# DicoCo

*Quentin LACHAUSSEE*

In [25]:
import pandas as pd
import requests
import pandas as pd
import re
from datetime import datetime
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Markdown 
import requests
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from requests_html import HTMLSession
import os.path

In [27]:
chemin_Buchanan_riche = "./../data/dataframe/BuchananRich.csv"
df_Buchanan_riche = pd.read_csv(chemin_Buchanan_riche, sep=";")

filter_hwd = widgets.Text(placeholder='.*',value='^a',description="filtre hwd")
filter_ipa = widgets.Text(placeholder='.*',value='',description="filtre ipa")
filter_occ = widgets.IntSlider(value=3, min=1, max=10, step=1, description='occ. Max:')

out = widgets.Output()

def filter_df(df, str_hwd, str_ipa, int_occ):    
    return df[(df['motVedette'].str.contains(str_hwd, na=False,flags=re.IGNORECASE, regex=True)) &
              (df['pronIpa'].str.contains(str_ipa, na=False,flags=re.IGNORECASE, regex=True))].head(int_occ).reset_index(drop=True)

def save_load_mp3(link, word, langage):
    path_png = f"./../data/audio/song_{word}_{langage}.png"
    
    if os.path.exists(path_png) == False:
        doc = requests.get(link)
        with open(f"./../data/audio/song.mp3", 'wb') as f:
            f.write(doc.content)
        path = f"./../data/audio/song.mp3"
        y, sr = librosa.load(path)
        D = librosa.stft(y)  # STFT of y
        S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
        fig, ax = plt.subplots()
        img = librosa.display.specshow(S_db, x_axis='time', y_axis='linear', ax=ax)
        fig.colorbar(img, ax=ax, format="%+2.f dB")
        plt.savefig(path_png)

    return path_png

def on_button_clicked(_):
    with out:
        
        clear_output()

        str_hwd = filter_hwd.value
        str_ipa = filter_ipa.value
        int_occ = int(filter_occ.value)
        if str_hwd=="":
            str_hwd = ".*"
        else:
            pass
        if str_ipa=="":
            str_ipa = ".*"
        else:
            pass

        df_filtered = filter_df(df_Buchanan_riche, str_hwd, str_ipa, int_occ)
        
        tableau_label = '<table><tr><th style="border: 1px solid black; text-align: center;">motVedette</th>\
                                    <th style="border: 1px solid black; text-align: center;">nbSyll</th>\
                                    <th style="border: 1px solid black; text-align: center;">pronOrigin</th>\
                                    <th style="border: 1px solid black; text-align: center;">pronSampa</th>\
                                    <th style="border: 1px solid black; text-align: center;">pronPos</th>\
                                    <th style="border: 1px solid black; text-align: center;">pronIpa</th>\
                                    <th style="border: 1px solid black; text-align: center;">US</th>\
                                    <th style="border: 1px solid black; text-align: center;">UK</th>\
                                </tr>'
        for i in range(len(df_filtered)):
            word = df_filtered.motVedette[i]
            url = f"https://www.linguee.fr/anglais-francais/traduction/{word}.html"
            response = requests.get(url)
            if response.status_code == 503:
                print("Nombre de requête dépassé")
                break
            body = BeautifulSoup(response.text, 'html.parser').find('body')
            span = body.find('span',{"class":"tag_lemma"})
            audio_US = span.find('a',{"class":"audio"})['onclick'].split('"')[1]
            audio_UK = span.find('a',{"class":"audio"})['onclick'].split('"')[5]
            audio_US_url = f"https://www.linguee.fr/mp3/{audio_US}.mp3"
            audio_UK_url = f"https://www.linguee.fr/mp3/{audio_UK}.mp3"
            audio_US_plot = save_load_mp3(audio_US_url, word, "us")
            audio_UK_plot = save_load_mp3(audio_UK_url, word, "uk")

            tableau_label += '<tr><td rowspan="2" style="border: 1px solid black; text-align: center;"><a href='+url+' target="_blank" >'+word+'</a></td>\
                                  <td rowspan="2" style="border: 1px solid black; text-align: center;">'+str(df_filtered.nbSyll[i])+'</td>\
                                  <td rowspan="2" style="border: 1px solid black; text-align: center;">'+df_filtered.pron[i]+'</td>\
                                  <td rowspan="2" style="border: 1px solid black; text-align: center;">'+df_filtered.pronSampa[i]+'</td>\
                                  <td rowspan="2" style="border: 1px solid black; text-align: center;">'+df_filtered.pronPos[i]+'</td>\
                                  <td rowspan="2" style="border: 1px solid black; text-align: center;">'+df_filtered.pronIpa[i]+'</td>\
                                  <td style="border: 1px solid black; border-bottom: 0px;"> <audio controls src="'+audio_US_url+'"><code>audio</code></audio></td>\
                                  <td style="border: 1px solid black; border-bottom: 0px;"> <audio controls src="'+audio_UK_url+'"><code>audio</code></audio></td>\
                              </tr>\
                              <tr><td style="border: 1px solid black; border-top: 0px;"> <img src="'+audio_US_plot+'"></img></td>\
                                  <td style="border: 1px solid black; border-top: 0px;"> <img src="'+audio_UK_plot+'"></img></td>\
                              </tr>'


        tableau_label += '</table>'
        tableau = HTML(tableau_label)
        display(tableau)


button = widgets.Button(description='Collecter')    
button.on_click(on_button_clicked)

input_filter = widgets.HBox([filter_hwd, filter_ipa, filter_occ, button])

output_filter = widgets.VBox([out])

all_widget = widgets.VBox([input_filter, output_filter])

display(all_widget)
